# <font color=purple>Interpolación</font> 

In [13]:
using Plots
plotly()
datax = [0.15,2.3,3.15,4.85,6.25,7.95]
datay = [4.79867,4.49013,4.2243,3.47313,2.66674,1.511909];

In [4]:
scatter(datax,datay,xlabel="x",ylabel="y",color="green",
        grid=false,legend=false)

## <font color =blue>Polinomio de Lagrange</font>

In [9]:
function lg(datax::Array{Float64},datay::Array{Float64},pa::Array{Float64})
  m = length(pa) 
  n = length(datax)  
  L = ones(m,n)
  for j ∈ 1:n
        for k ∈ 1:n 
              if k!=j
                 L[:,j] .*= ( pa - datax[k] )/( datax[j] - datax[k] )
              end
        end
  end        
  return L*datay
 end

lg (generic function with 1 method)

In [10]:
mx = minimum(datax)
Mx = maximum(datax)
px = collect(linspace(mx,Mx,101))
py = lg(datax,datay,px)
plot!(px,py,title="Polinomio Interpolante L(x)")

## <font color=blue>Polinomio de Newton</font>

En la base de Newton, el polinomio interpolante para los puntos $(x_1,y_1),\dots,(x_6,y_6)$ es

$$\begin{array}{lll}
p(x) = &  & f(x_1) \\
     & + & f[x_1,x_2](x-x_1) \\
     & + & f[x_1,x_2,x_3](x-x_1)(x-x_2) \\
     & + & f[x_1,x_2,x_3,x_4](x-x_1)(x-x_2)(x-x_3) \\
     & + & f[x_1,x_2,x_3,x_4,x_5](x-x_1)(x-x_2)(x-x_3)(x-x_4) \\
     & + & f[x_1,x_2,x_3,x_4,x_5,x_6](x-x_1)(x-x_2)(x-x_3)(x-x_4)(x-x_5)
\end{array}$$

Creamos la rutina **NDD** para obtener las diferencias divididas

La idea es almacenar las diferencias divididas en la parte triangular inferior de
una matriz

$$\quad A=\left(\begin{array}{llllll}
 f(x_1) & & & & & \\ 
 f(x_2) & f[x_1,x_2] & & & &\\ 
 f(x_3) & f[x_2,x_3] & f[x_1,x_2,x_3] & & & \\ 
 f(x_4) & f[x_3,x_4] & f[x_2,x_3,x_4] & f[x_1,x_2,x_3,x_4] & & \\ 
 f(x_5) & f[x_4,x_5] & f[x_3,x_4,x_5] & f[x_2,x_3,x_4,x_5] &   
 f[x_1,x_2,x_3,x_4,x_5] & \\ 
 f(x_6) & f[x_5,x_6] & f[x_4,x_5,x_6] & f[x_3,x_4,x_5,x_6] &  
 f[x_2,x_3,x_4,x_5,x_6] & f[x_1,x_2,x_3,x_4,x_5,x_6] \\ 
\end{array}\right)$$

columna $\begin{array}{llllll}
\hspace{1.0cm} 1 & \hspace{1.0cm} 2 & \hspace{1.5cm} 3 & \hspace{2.0cm} 4 & \hspace{2.5cm} 5 & \hspace{3.0cm} 6 
\end{array}$

Para almacenar, observe 

$$f[x_3,x_4,x_5,x_6] = \dfrac{f[x_4,x_5,x_6]-f[x_3,x_4,x_5]}{x_6-x_3} \qquad\Longrightarrow\qquad
a_{6,4}= \dfrac{a_{6,3}-a_{5,3}}{x_6-x_3}$$

En este caso $i=6,j=4$. Entonces

$$a_{i,j}= \dfrac{a_{i,j-1}-a_{i-1,j-1}}{x_i-x_{i-j+1}}$$

Esta es la iteración principal 

In [1]:
function NDD(x::Array{Float64},y::Array{Float64})
    n = length(x)
    #crea una matriz donde en la parte triangular inferior almacena
    #las diferencias divididas
    A = zeros(n,n)
    # primera columna son evaluaciones 
    A[:,1]= y
    # en la columna j poner las diferencias divididas de orden j-1
    for j ∈ 2:n
        # llena la parte triangular inferior
        for i ∈ j:n
            # iteración principal
            A[i,j] = (A[i,j-1]-A[i-1,j-1]) / (x[i]-x[i-j+1])
        end
    end
    #regresa las diferencias divididas sobre la diagonal principal 
    #de la matriz, estas son coeficientes del polinomio de Newton 
    return diag(A)
end

NDD (generic function with 1 method)

In [5]:
println("coeficientes")
strf = "f[x_1]"
a = NDD(datax,datay)
for i ∈ 1:length(a)
    if i ==1
        println("$strf = $(a[i])")
    else
        l = length(strf)
        sl = strf[1:l-1]
        si = string(i)
        strf = "$sl,x_$si]"
        println("$strf = $(a[i])")
    end
end

coeficientes
f[x_1] = 4.79867
f[x_1,x_2] = -0.1435069767441864
f[x_1,x_2,x_3] = -0.05641139990880034
f[x_1,x_2,x_3,x_4] = 0.0012286641932144312
f[x_1,x_2,x_3,x_4,x_5] = 0.00010443283127582936
f[x_1,x_2,x_3,x_4,x_5,x_6] = -8.742359956899587e-6


Creamos rutina **polyNewton** para evaluar el polinomio interpolante de Newton 

La idea es sobreescribir el polinomio mediante una relación de recurrencia 
(*método de Horner*)

$$\begin{array}{ll}
 p(x) & \longleftarrow f[x_1,x_2,x_3,x_4,x_5,x_6] \\
 p(x) & \longleftarrow (x-x_5)p(x) + f[x_1,x_2,x_3,x_4,x_5] \\
 p(x) & \longleftarrow (x-x_4)p(x) + f[x_1,x_2,x_3,x_4] \\
 p(x) & \longleftarrow (x-x_3)p(x) + f[x_1,x_2,x_3] \\
 p(x) & \longleftarrow (x-x_2)p(x) + f[x_1,x_2] \\
 p(x) & \longleftarrow (x-x_1)p(x) + f(x_1) \\
\end{array}$$

Esto nos da el polinomio en la base de Newton

In [6]:
#Rutina que evalua el polinomio de Newton
#Entrada: arreglo pa con partición del intervalo min(x_i)<=x<=max(x_i)
#         arreglo nod de nodos
#         arreglo c con coeficientes(diferencias divididas) 
#Salida:  arreglo px con evaluaciones del polinomio en particion pa
function polyNewton(pa,nod,c)
    n = length(nod) -1
    px = c[n]
    for i ∈ 1:n-1
        px = px.*(pa-nod[n-i]) + c[n-i]
    end
    return px
end

polyNewton (generic function with 1 method)

In [11]:
tval = linspace(mx,Mx,60)
yval = polyNewton(tval,datax,a)
scatter(datax,datay,xlabel="x",legend=false,grid=false,color="green")
plot!(tval,yval,title="polinomio interpolante de Newton p(x)")

# <font color=purple>Reglas Compuestas de Cuadratura</font> 

In [ ]:
año = linspace(1997,2005,9)
edad = [34.7,34.9,35.2,35.3,35.6,35.7,35.9,36,36.2]
println(" año     edad ")
for i ∈ 1:length(año)
    println("$(año[i])   $(edad[i])")
end

In [ ]:
using Plots
plotly()

In [ ]:
scatter(año,edad,grid=false,legend=false,title="Edad Media",xlabel="año",
        ylabel="edad")


## <font color=blue>Regla compuesta del Trapecio</font>

- 8 subintervalos  $\longrightarrow$ 8 segmentos de recta $l_i(t)$


$$\text{edad media}\quad f(t) \approx
 \begin{cases}
  l_1(t) & 1997 \leq t  \leq 1998,\\
  l_2(t) & 1998 \leq t  \leq 1999,\\
  l_3(t) & 1999 \leq t  \leq 2000,\\
  l_4(t) & 2000 \leq t  \leq 2001,\\
  l_5(t) & 2001 \leq t  \leq 2002,\\
  l_6(t) & 2002 \leq t  \leq 2003,\\
  l_7(t) & 2003 \leq t  \leq 2004,\\
  l_8(t) & 2004 \leq t  \leq 2005,\\
 \end{cases}$$

In [ ]:
plot(año[1:2],edad[1:2],color ="#5760e3",label="l₁(t)",linewidth=6,fillcolor="blue")
plot!(año[2:3],edad[2:3],color ="#15f2f9",label="l₂(t)",linewidth=6)
plot!(año[3:4],edad[3:4],color ="#608cf7",label="l₃(t)",linewidth=6)
plot!(año[4:5],edad[4:5],color ="#631a85",label="l₄(t)",linewidth=6)
plot!(año[5:6],edad[5:6],color ="#051938",label="l₅(t)",linewidth=6)
plot!(año[6:7],edad[6:7],color ="#7689a9",label="l₆(t)",linewidth=6)
plot!(año[7:8],edad[7:8],color ="#4baef9",label="l₇(t)",linewidth=6)
plot!(año[8:9],edad[8:9],color ="#b700ff",label="l₈(t)",linewidth=6,grid=false)  

Aproximación de Edad promedio $\int_{1997}^{2005} f(t)dt$

In [ ]:
plot(año[1:2],edad[1:2],fill=(0,"#5760e3"),label="l₂(t)")
plot!(año[2:3],edad[2:3],fill=(0,"#15f2f9"),label="l₂(t)")
plot!(año[3:4],edad[3:4],fill=(0,"#608cf7"),label="l₃(t)")
plot!(año[4:5],edad[4:5],fill=(0,"#631a85"),label="l₄(t)")
plot!(año[5:6],edad[5:6],fill=(0,"#051938"),label="l₅(t)")
plot!(año[6:7],edad[6:7],fill=(0,"#7689a9"),label="l₆(t)")
plot!(año[7:8],edad[7:8],fill=(0,"#4baef9"),label="l₇(t)")
plot!(año[8:9],edad[8:9],fill=(0,"#b700ff"),label="l₈(t)")

$$\begin{array}{c}
\text{área bajo segmento de recta} \\ 
\text{que une nodo $i$ con nodo $i+1$} 
\end{array} 
\quad = \quad
\begin{array}{c}
  \text{área de trapecio} \\
  \text{altura = incremento, base 1 = $f$(nodo $i$) y base 2 = $f$(nodo $i+1$)}
\end{array}$$

$$\begin{array}{c} \\ \end{array}$$

$$ \text{Área de un trapecio} 
= \text{altura}\times\dfrac{\text{base 1} + \text{base 2}}{2}$$

$$\begin{array}{c} \\ \end{array}$$

En ejemplo, todas las alturas = 1

Regla compuesta a partir de reglas simples:

$$\begin{array}{ll}
\int_{1997}^{1998} l_1(t)dt & = \dfrac{34.7+34.9}{2} \\
\int_{1998}^{1999} l_2(t)dt & = \dfrac{34.9+35.2}{2}\\
\int_{1999}^{2000} l_3(t)dt & = \dfrac{35.2+35.3}{2} \\
\int_{2000}^{2001} l_4(t)dt & = \dfrac{35.3+35.6}{2}\\
\int_{2001}^{2002} l_5(t)dt & = \dfrac{35.6+35.7}{2}\\
\int_{2002}^{2003} l_6(t)dt & = \dfrac{35.7+35.9}{2}\\
\int_{2003}^{2004} l_7(t)dt & = \dfrac{35.9+36}{2}\\
\int_{2004}^{2005} l_8(t)dt & = \dfrac{36+36.2}{2}
\end{array}$$

 $$\begin{array}{ll}
 \int_{1997}^{2005} f(t)dt  & \approx
 \dfrac{1}{2}[34.7 + (2\times34.9)+ (2\times 35.2) + (2\times 35.3) \\
 & \qquad + (2\times 35.6) + (2\times35.7) + (2\times35.9) + (2\times36) +36.2]
 \end{array}$$

In [ ]:
Ia = .5*(34.7 + 2*34.9 + 2*35.2 + 2*35.3 + 2*35.6 + 2*35.7 + 2*35.9 + 2*36 + 36.2)
println("∫₁₉₉₇²⁰⁰⁵  f(t)dt ≈",Ia)

**Regla compuesta del Trapecio**
$$\begin{array}{ll}  
& =  \dfrac{incremento}{2}\left[\right. 
f(1º\ nodo) + 2f(2º\ nodo) + 2f(3º\ nodo) + 2f(4º\ nodo)  \\ 
 & + 2f(5º\ nodo) + 2f(6º\ nodo) + 2f(7º\ nodo) + 2f(8º\ nodo) + f(9º\ nodo)\left.\right]
 \end{array}$$